In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "akhilkathi97"
os.environ['KAGGLE_KEY'] = "b1a7ecbde49a15f7d8c855d7171c1f61"

In [ ]:
!pip install --upgrade gensim --pre
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews --force
!7z e imdb-dataset-of-50k-movie-reviews.zip -y
!pip install contractions -q
!gdown --id 1pGd5tLwA30M7wkbJKdXHaae9tYVDICJ_

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
 19% 5.00M/25.7M [00:00<00:00, 45.9MB/s]
100% 25.7M/25.7M [00:00<00:00, 142MB/s] 

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 26962657 bytes (26 MiB)

Extracting archive: imdb-dataset-of-50k-movie-reviews.zip
--
Path = imdb-dataset-of-50k-movie-reviews.zip
Type = zip
Physical Size = 26962657

  0%     26% - IMDB Dataset.csv                        50% - IMDB Dataset.csv            

In [ ]:
import contractions 
import nltk
import re
from tabulate import tabulate
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob
import numpy as np 
import pandas as pd
import csv
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier,RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import FastText,Word2Vec
import time
import pickle
from sklearn.model_selection import train_test_split,KFold
import warnings
warnings.filterwarnings("error")
warnings.filterwarnings("ignore", category=DeprecationWarning)
tqdm.pandas()
import xgboost as xgb
from scipy import stats
from sklearn.naive_bayes import MultinomialNB
import multiprocessing

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
cpu_count = multiprocessing.cpu_count()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
stopwords_en = stopwords.words('english')
stemmerSnowball = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
def text_preprocessing(text):
    #Removing Hashtags
    text = re.sub("@[A-Za-z0-9_]+","", text)
    #Removing Mentions
    text = re.sub("#[A-Za-z0-9_]+","", text)
    #Remove <br/> tags
    text = re.sub(r"\<.*\/\>",'',text)
    #Remove emojis
    text=text.encode("ascii","ignore")
    text=text.decode()
    #Remove words containing numbers/digits
    text = re.sub(r'\w*\d\w*',"",text)
    #removing certain sized words
    text=' '.join([i for i in text.split() if len(i)>2])
    #Remove HTML tags
    remove_html_tags = [re.sub(r"\<.*\>",'',word) for word in text.split(" ") ]
    #Remove words inside brackets
    remove_bracket_words = [re.sub(r"\(.*\)",'',word) for word in remove_html_tags ]
    #Expanding contractions
    text = contractions.fix((str(" ".join(remove_bracket_words)).lower()))
    #Words
    words = [re.sub(r'[^\w\s]','',word) for word in text.split(' ')]
    #Correcting Spelling Mistakes
    #correct_words = [parser.parse(_)['result'] for _ in words]
    #Removing StopWords
    without_stop_words =  [re.sub(r'[^\w+]','',word) for word in words if word not in stopwords_en]
    #Stemming
    stemmedWords =  [stemmerSnowball.stem(word) for word in without_stop_words]
    #Lemmatization
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmedWords]
    #Tokenization
    tokenized = [" ".join(tokenizer.tokenize(word)) for word in lemmatized]

    return re.sub(' +', ' ',str(" ".join(tokenized)))

In [ ]:
data = pd.read_csv('IMDB Dataset.csv')
data['pre_processed_text'] = data['review'].progress_apply(lambda row:text_preprocessing(row))

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
X = data['pre_processed_text']
data['sentiment'] =  data['sentiment'].apply(lambda row: 0 if row=="positive" else 1)
y = data['sentiment']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25, random_state=42,stratify = y)

In [ ]:
pre_processed_text_list_train =  X_train.apply(lambda x: x.split())
pre_processed_text_list_test =  X_test.apply(lambda x: x.split())
pre_processed_sentences_list_train = pre_processed_text_list_train.tolist()
pre_processed_sentences_list_test = pre_processed_text_list_test.tolist()

[Reference 1](https://www.geeksforgeeks.org/fasttext-working-and-implementation/)
[Reference 2](https://towardsdatascience.com/fasttext-under-the-hood-11efc57b2b3)

1.   fastText is deep learning model which uses either continuous bag of words, skipgrams or bag of words architecture in the input. 
2.   At output layer as opposed to a linear classifier with softmax activation with gradient descent implementation, a hierarchical classifier is used.
3.   A hierarchical classifier uses huffman algorithm to build decision trees. Every node in the binary tree represents a probability. A label is represented by the probability along the path to that given label.
4.   Since the model will only be calculating only the probability on each node in the path to the one correct label, this method vastly reduces the time complexity of training the model.
5.   fastText models using negative sampling, softmax or hierarchical softmax loss functions.
6.   fastText performs well on out of vocabulary words since there is high change of word being an n-gram from existing corpus or complex word vectors are sometimes obtained by summing up two known simple words which form a complex word.




  

In [ ]:
reviews_model = FastText(vector_size=300, window=2)
reviews_model.build_vocab(pre_processed_sentences_list_train)
start_time_fast_text = time.time()
reviews_model.train(pre_processed_sentences_list_train,total_examples=len(pre_processed_sentences_list_train),epochs=20)
end_time_fast_text = time.time()
print(f"Training Time: {end_time_fast_text - start_time_fast_text}s")

Training Time: 224.7142312526703s


In [ ]:
!gdown --id 1pGd5tLwA30M7wkbJKdXHaae9tYVDICJ_

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1pGd5tLwA30M7wkbJKdXHaae9tYVDICJ_ 



In [ ]:
words = reviews_model.wv.index_to_key
print(len(words))

18773


In [ ]:
#https://www.kaggle.com/code/tarushi89/nlp-tutorial-word2vec-glove-embeddings

def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float64")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index_to_key)
    
    for word in  words.split():
        if word in index2word_set:
            #print("Found Word")
            nwords = nwords + 1
            featureVec = np.add(featureVec,model.wv[word])
    
    # Dividing the result by number of words to get average
    try:
      featureVec = np.divide(featureVec, nwords)
    except RuntimeWarning:
      print(np.divide(featureVec, nwords))
    finally:
      return featureVec

In [ ]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float64")
    for review in tqdm(reviews):
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
    return reviewFeatureVecs

In [ ]:
trainVectors = getAvgFeatureVecs(X_train,reviews_model,300)
testVectors = getAvgFeatureVecs(X_test,reviews_model,300)

  0%|          | 0/37500 [00:00<?, ?it/s]

  0%|          | 0/12500 [00:00<?, ?it/s]

In [ ]:
kfold_5 = KFold(shuffle = True, n_splits = 5)

In [ ]:
dict_params_dtc = dict(max_features=["sqrt","log2"],criterion = ['gini', 'entropy'])
dtc_random_search = RandomizedSearchCV(DecisionTreeClassifier(),dict_params_dtc,random_state=0,n_iter=4,scoring ='accuracy',n_jobs=-1,cv=kfold_5)
dtc_rcv_search = dtc_random_search.fit(trainVectors,y_train)
dtc_best_param_table = tabulate([list(dtc_rcv_search.best_params_.values())], headers=list(dtc_rcv_search.best_params_.keys()), tablefmt="grid")
dtc_best_params = DecisionTreeClassifier(**dtc_rcv_search.best_params_)
dtc_best_params.fit(trainVectors,y_train)
dtc_y_pred = dtc_best_params.predict(testVectors)
dtc_accuracy = dtc_best_params.score(testVectors,y_test)
dtc_report = classification_report(y_test,dtc_y_pred, output_dict=True)

In [ ]:
dict_params_rfc = dict(max_features=["sqrt","log2"],criterion = ['gini', 'entropy'])
rfc_random_search = RandomizedSearchCV(RandomForestClassifier(),dict_params_rfc,random_state=0,n_iter=4,scoring ='accuracy',n_jobs = -1,cv=kfold_5)
rfc_rcv_search = rfc_random_search.fit(trainVectors,y_train)
rfc_best_param_table = tabulate([list(rfc_rcv_search.best_params_.values())], headers=list(rfc_rcv_search.best_params_.keys()), tablefmt="grid")
rfc_best_params = RandomForestClassifier(**rfc_rcv_search.best_params_)
rfc_best_params.fit(trainVectors,y_train)
rfc_y_pred = rfc_best_params.predict(testVectors)
rfc_accuracy = rfc_best_params.score(testVectors,y_test)
rfc_report = classification_report(y_test,rfc_y_pred,output_dict=True)

In [ ]:
dict_params_svc = dict(kernel=["linear","rbf"],C=[0.01,0.1])
svc_random_search = RandomizedSearchCV(SVC(),dict_params_svc,random_state=0,n_iter=4,scoring ='accuracy',cv=kfold_5)
svc_rcv_search = svc_random_search.fit(trainVectors,y_train)
svc_best_param_table = tabulate([list(svc_rcv_search.best_params_.values())], headers=list(svc_rcv_search.best_params_.keys()), tablefmt="grid")
svc_best_params = SVC(**svc_rcv_search.best_params_)
svc_best_params.fit(trainVectors,y_train)
svc_y_pred = svc_best_params.predict(testVectors)
svc_accuracy = svc_best_params.score(testVectors,y_test)
svc_report = classification_report(y_test,svc_y_pred,output_dict=True)

In [ ]:
dict_params_xgb = {'n_estimators': [300,750,1000],'learning_rate':[0.01,0.05,0.9],'max_depth': [5,6,7],'objective': ['binary:logistic']}
clf_xgb = xgb.XGBClassifier(tree_method='gpu_hist')
xgb_random_search = RandomizedSearchCV(clf_xgb,dict_params_xgb,random_state=0,cv=kfold_5,n_iter=3,scoring='accuracy',n_jobs=-1)
xgb_rcv_search = xgb_random_search.fit(trainVectors,y_train)
xgb_best_param_table = tabulate([list(xgb_rcv_search.best_params_.values())], headers=list(xgb_rcv_search.best_params_.keys()), tablefmt="grid")
xgb_best_model_params = xgb_rcv_search.best_params_
xgb_best_model_params['tree_method']='gpu_hist'
xgb_best_params =  xgb.XGBClassifier(**xgb_best_model_params)
xgb_best_params.fit(trainVectors,y_train)
xgb_accuracy = xgb_best_params.score(testVectors,y_test)
xgb_report = classification_report(y_test,xgb_best_params.predict(testVectors),output_dict=True)

In [ ]:
print('Best Parameters')
print(tabulate([[dtc_best_param_table,rfc_best_param_table,svc_best_param_table,xgb_best_param_table]], headers=['Decision Trees','Random Forest','Support Vectors','Gradient Boosting'], tablefmt="grid"))

Best Parameters
+----------------------------------+----------------------------------+--------------------+----------------------------------------------------------------------+
| Decision Trees                   | Random Forest                    | Support Vectors    | Gradient Boosting                                                    |
+==================================+==================================+====================+======================================================================+
| +----------------+-------------+ | +----------------+-------------+ | +----------+-----+ | +-----------------+----------------+-------------+-----------------+ |
| | max_features   | criterion   | | | max_features   | criterion   | | | kernel   |   C | | | objective       |   n_estimators |   max_depth |   learning_rate | |
| +================+=============+ | +================+=============+ | +==========+=====+ | +=================+================+=============+=================+ |


In [ ]:
estimators = [('dtc',DecisionTreeClassifier(**dtc_rcv_search.best_params_)),('rfc',RandomForestClassifier(**rfc_rcv_search.best_params_)),('svc',SVC(**svc_rcv_search.best_params_)),('xgb',xgb.XGBClassifier(**xgb_best_model_params))]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(),n_jobs=-1)
stacking_accuracy = clf.fit(trainVectors,y_train).score(testVectors, y_test)

In [ ]:
table = []
headers = list(list(dtc_report.values())[0].keys())
classification_reports_ = [dtc_report,rfc_report,svc_report,xgb_report]
for report in classification_reports_:
  sub_table = []
  for idx,_ in enumerate(list(report.values())):
    try:
      _vals = list(_.values())
      _vals.insert(0,list(report.keys())[idx])
      sub_table.append(_vals)
    except:
      pass
  table.append(sub_table)

In [ ]:
dtc_report_table = tabulate(table[0], headers=headers, tablefmt="grid")
rfc_report_table = tabulate(table[1], headers=headers, tablefmt="grid")
svc_report_table = tabulate(table[2], headers=headers, tablefmt="grid")
xgb_report_table = tabulate(table[3], headers=headers, tablefmt="grid")

In [ ]:
print('Classification Reports')
print(tabulate([[dtc_report_table,rfc_report_table,svc_report_table,xgb_report_table]], headers=['Decision Trees','Random Forest','Support Vectors','Extreme Gradient Boosting'], tablefmt="grid"))

Classification Reports
+--------------------------------------------------------------------+--------------------------------------------------------------------+--------------------------------------------------------------------+--------------------------------------------------------------------+
| Decision Trees                                                     | Random Forest                                                      | Support Vectors                                                    | Extreme Gradient Boosting                                          |
+====================================================================+====================================================================+====================================================================+====================================================================+
| +--------------+-------------+----------+------------+-----------+ | +--------------+-------------+----------+------------+-----------+ | +--

In [ ]:
print('Model Scores')
print(tabulate([[dtc_accuracy,rfc_accuracy,svc_accuracy,xgb_accuracy,stacking_accuracy]], headers=['Decision Trees','Random Forest','Support Vectors','Extreme Gradient Boosting','Stacking Algorithm'], tablefmt="grid"))

Model Scores
+------------------+-----------------+-------------------+-----------------------------+----------------------+
|   Decision Trees |   Random Forest |   Support Vectors |   Extreme Gradient Boosting |   Stacking Algorithm |
+==================+=================+===================+=============================+======================+
|          0.64416 |         0.78536 |           0.82904 |                     0.82688 |              0.82816 |
+------------------+-----------------+-------------------+-----------------------------+----------------------+


In [ ]:
word_to_vec = Word2Vec(pre_processed_sentences_list_train, vector_size=300,window=5, min_count=2)
trainVectors_word_to_vec = getAvgFeatureVecs(X_train,word_to_vec,300)
testVectors_word_to_vec = getAvgFeatureVecs(X_test,word_to_vec,300)

  0%|          | 0/37500 [00:00<?, ?it/s]

  0%|          | 0/12500 [00:00<?, ?it/s]

In [ ]:
estimators = [('dtc',DecisionTreeClassifier(**dtc_rcv_search.best_params_)),('rfc',RandomForestClassifier(**rfc_rcv_search.best_params_)),('svc',SVC(**svc_rcv_search.best_params_)),('xgb',xgb.XGBClassifier(**xgb_best_model_params))]
clf_word_to_vec = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(),n_jobs=-1)
word_to_vec_acc = clf_word_to_vec.fit(trainVectors_word_to_vec,y_train).score(testVectors_word_to_vec, y_test)

In [ ]:
tf_idf = TfidfVectorizer(ngram_range=(1,2), max_df=0.95, min_df=2, max_features=2500).fit(X_train,y_train)
tf_idf_train = tf_idf.transform(X_train).toarray()
tf_idf_test = tf_idf.transform(X_test).toarray()

In [ ]:
estimators = [('dtc',DecisionTreeClassifier(**dtc_rcv_search.best_params_)),('rfc',RandomForestClassifier(**rfc_rcv_search.best_params_)),('svc',SVC(**svc_rcv_search.best_params_)),('xgb',xgb.XGBClassifier(**xgb_best_model_params))]
tf_idf_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(),n_jobs=-1)
tf_idf_acc = tf_idf_clf.fit(tf_idf_train,y_train).score(tf_idf_test, y_test)

In [ ]:
print('Different Word Embeddings')
print(tabulate([[stacking_accuracy,word_to_vec_acc,tf_idf_acc]], headers=['fastText Word Embeddings','Average Word to Vector','TF-IDF'], tablefmt="grid"))

Different Word Embeddings
+----------------------------+--------------------------+----------+
|   fastText Word Embeddings |   Average Word to Vector |   TF-IDF |
+============================+==========================+==========+
|                    0.82608 |                  0.82104 |  0.84528 |
+----------------------------+--------------------------+----------+
